## Step 1: Environment Setup

In [ ]:
pip install --upgrade openai


In [ ]:
# Part 1: Environment Setup
!pip install openai textblob matplotlib psycopg2


## Step 2: User Interaction and GPT-3 Response Generation

In [ ]:
import openai
import os
import matplotlib.pyplot as plt
from textblob import TextBlob

# Set OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY", "your-api-key-here")

# Function to generate a GPT-3 response based on user input
def generate_response(user_input):
    try:
        response = openai.Completion.create(
            model="text-davinci-003",
            prompt=f"User: {user_input}\nGPT-3:",  # Prompt for GPT-3 response
            max_tokens=50,
            temperature=0.7,
        )
        return response.choices[0].text.strip()
    except Exception as e:  # Catch all exceptions
        print("An error occurred while generating response:", e)
        return None

# User interaction function with updated error handling
def user_interaction():
    """Interact with the user and generate responses."""
    conversation = []
    user_input = ""
    
    while user_input.lower() not in ["exit", "quit"]:
        try:
            # Prompt for user input
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                print("Goodbye!")
                break
            
            # Generate GPT-3 response
            response = generate_response(user_input)
            
            if response:  # Check if response is not None
                # Display and store the response
                print("GPT-3:", response)
                conversation.append({
                    "user_input": user_input,
                    "gpt_response": response,
                })
        except Exception as e:  # Catch all unexpected errors
            print("Unexpected error during user interaction:", e)

    return conversation  # Return the complete conversation


## Step 3: Sentiment Analysis and Data Visualization

In [ ]:
# Part 4: Sentiment Analysis
def analyze_sentiment(text):
    """Perform sentiment analysis using TextBlob."""
    try:
        analysis = TextBlob(text)
        polarity = analysis.sentiment.polarity
        if polarity > 0:
            return "positive"
        elif polarity < 0:
            return "negative"
        else:
            return "neutral"
    except Exception as e:
        print("Error analyzing sentiment:", e)
        return "unknown"

# Function to analyze sentiment for each response in a conversation
def sentiment_analysis(conversation):
    if not conversation:
        print("No conversation data available for sentiment analysis.")
        return None
    
    sentiment_results = {
        "positive": 0,
        "negative": 0,
        "neutral": 0,
    }
    
    for entry in conversation:
        sentiment = analyze_sentiment(entry["gpt_response"])
        sentiment_results[sentiment] += 1
        entry["sentiment"] = sentiment  # Add sentiment info
    
    return sentiment_results


In [ ]:
# Part 5: Data Visualization and Sentiment Report
def create_sentiment_report(sentiment_results):
    if sentiment_results is None:
        print("No sentiment results to visualize.")
        return
    
    labels = list(sentiment_results.keys())
    sizes = list(sentiment_results.values())
    colors = ["green", "red", "gray"]  # Colors for positive, negative, neutral
    explode = (0.1, 0, 0)  # Highlight positive sentiment
    
    plt.pie(sizes, explode=explode, labels=labels, colors=colors,
            autopct="%1.1f%%", shadow=True, startangle=140)
    plt.axis("equal")  # Maintain circular shape
    plt.title("Sentiment Analysis Distribution")
    plt.show()


## Step 4: PostgreSQL Database Integration

In [ ]:
# Part 6: PostgreSQL Database Integration
def connect_db():
    """Connect to PostgreSQL database."""
    try:
        conn = psycopg2.connect(
            host="localhost",
            port="5432",
            user="your_db_user",
            password="your_db_password",
            dbname="sentiment_db",
        )
        return conn
    except Exception as e:
        print("Database connection error:", e)
        return None

# Function to store conversation in the database
def store_conversation_in_db(conversation):
    conn = connect_db()
    if conn is None:
        print("Unable to connect to the database.")
        return
    
    cur = conn.cursor()
    
    # Create the table if it doesn't exist
    cur.execute("""
        CREATE TABLE IF NOT EXISTS conversations (
            id SERIAL PRIMARY KEY,
            user_input TEXT,
            gpt_response TEXT,
            sentiment TEXT
        );
    """)
    
    # Insert the conversation into the database
    for entry in conversation:
        cur.execute("""
            INSERT INTO conversations (user_input, gpt_response, sentiment)
            VALUES (%s, %s, %s);
        """, (entry["user_input"], entry["gpt_response"], entry["sentiment"]))
    
    conn.commit()
    cur.close()
    conn.close()


In [ ]:
# Main Execution: Interaction, Sentiment Analysis, and Database Storage
conversation = user_interaction()  # Properly define and initialize conversation
sentiment_results = sentiment_analysis(conversation)  # Analyze sentiment in conversation

# Create a sentiment report if results are available
if sentiment_results:  
    create_sentiment_report(sentiment_results)

# Store the conversation in the PostgreSQL database
store_conversation_in_db(conversation)


## Step 5: Docker Deployment

In [ ]:
# Part 7: Docker Deployment
dockerfile_content = """
# Base image for Python
FROM python:3.9-slim

# Set working directory
WORKDIR /app

# Install dependencies
COPY requirements.txt .
RUN pip install -r requirements.txt

# Copy the project files into the container
COPY . .

# Run Jupyter Notebook
CMD ["jupyter", "notebook", "--ip=0.0.0.0", "--port=8888", "--no-browser", "--allow-root"]
"""

with open("Dockerfile", "w") as f:
    f.write(dockerfile_content)

# Create a requirements.txt with all dependencies
requirements_content = """
openai
textblob
psycopg2
matplotlib
"""
with open("requirements.txt", "w") as f:
    f.write(requirements_content)
